In [1]:
import os
import re
from typing import Optional, List, Dict, Any
from deepeval.models import DeepEvalBaseLLM
from openai import OpenAI, AsyncOpenAI
from deepeval.models import DeepEvalBaseEmbeddingModel

In [2]:
class InfinityEmbeddingModel(DeepEvalBaseEmbeddingModel):
    def __init__(self, model_name: str, base_url: str, api_key: Optional[str] = None):
        self.model_name = model_name
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY", "123")
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None

    def load_model(self) -> OpenAI:
        if self._sync_client is None:
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._sync_client

    def load_async_model(self) -> AsyncOpenAI:
        if self._async_client is None:
            self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._async_client

    def embed_text(self, text: str) -> List[float]:
        client = self.load_model()
        try:
            response = client.embeddings.create(
                model=self.model_name,
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error during synchronous Infinity embedding for text '{text[:50]}...': {e}")
            return []

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        client = self.load_model()
        try:
            response = client.embeddings.create(
                model=self.model_name,
                input=texts
            )
            return [data.embedding for data in response.data]
        except Exception as e:
            print(f"Error during synchronous Infinity batch embedding: {e}")
            return [[] for _ in texts] 

    async def a_embed_text(self, text: str) -> List[float]:
        client = self.load_async_model()
        try:
            response = await client.embeddings.create(
                model=self.model_name,
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error during asynchronous Infinity embedding for text '{text[:50]}...': {e}")
            return []

    async def a_embed_texts(self, texts: List[str]) -> List[List[float]]:
        client = self.load_async_model()
        try:
            response = await client.embeddings.create(
                model=self.model_name,
                input=texts
            )
            return [data.embedding for data in response.data]
        except Exception as e:
            print(f"Error during asynchronous Infinity batch embedding: {e}")
            return [[] for _ in texts]


    def get_model_name(self) -> str:
        """
        Returns the name of the embedding model.
        """
        return self.model_name

In [ ]:
import os
import re
import json
from typing import Optional, Union, Dict, List, Any
from pydantic import BaseModel
import instructor
from openai import OpenAI, AsyncOpenAI
from deepeval.models import DeepEvalBaseLLM

class LLMModel(DeepEvalBaseLLM):
    COGITO_THINKING_INSTRUCTION = "Enable deep thinking subroutine."

    def __init__(self,
                 model_name: str,
                 base_url: str,
                 api_key: Optional[str] = None,
                 attempt_thinking_mode: bool = False,
                 cleaning_method: Optional[str] = None,
                 max_tokens: int = 2000
                ):
        self.model_name_original = model_name
        self.model_name_lower = model_name.lower()
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY", "EMPTY")
        # Используйте instructor.patch сразу при создании клиентов
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None
        self.attempt_thinking_mode = attempt_thinking_mode
        self.cleaning_method = cleaning_method
        self.max_tokens_to_generate = max_tokens
        super().__init__(model_name=model_name)

    def load_model(self) -> OpenAI:
        if self._sync_client is None:
            # Патчим клиент при создании
            self._sync_client = instructor.patch(OpenAI(base_url=self.base_url, api_key=self.api_key))
        return self._sync_client

    def load_async_model(self) -> AsyncOpenAI:
        if self._async_client is None:
            # Патчим асинхронный клиент при создании
            self._async_client = instructor.patch(AsyncOpenAI(base_url=self.base_url, api_key=self.api_key))
        return self._async_client

    def _clean_response(self, raw_response: str) -> str:
        if not raw_response:
            return ""

        cleaned = raw_response.strip()

        if self.cleaning_method == "rsplit":
            closing_tag = '</think>'
            if closing_tag in cleaned:
                parts = cleaned.rsplit(closing_tag, 1)
                think_block_start = parts[0].rfind('<think>')
                if think_block_start != -1:
                    cleaned = parts[0][:think_block_start].rstrip() + parts[1].lstrip()
                else:
                     cleaned = parts[1].lstrip()
            return cleaned.strip()

        elif self.cleaning_method == "regex":
            pattern = r'<think>.*?</think>\s*'
            cleaned = re.sub(pattern, '', cleaned, flags=re.DOTALL).strip()
            return cleaned
        else:
            return cleaned

    def _prepare_api_call_args(self, prompt: str) -> Dict[str, Any]:
        messages: List[Dict[str, str]] = [{"role": "user", "content": prompt}]
        api_extra_params: Dict[str, Any] = {}

        if self.attempt_thinking_mode:
            if "cogito" in self.model_name_lower:
                messages.insert(0, {"role": "system", "content": self.COGITO_THINKING_INSTRUCTION})

        return {"messages": messages, "api_extra_params": api_extra_params}

    # --- ИЗМЕНЕННЫЙ МЕТОД generate ---
    def generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Union[str, BaseModel]:  
        client = self.load_model()  
        call_args = self._prepare_api_call_args(prompt)  
        messages = call_args["messages"]  
        api_extra_params = call_args["api_extra_params"]  
    
        try:  
            # Если передана схема, используем response_model  
            if schema is not None:  
                response = client.chat.completions.create(  
                    model=self.model_name_original,  
                    messages=messages,  
                    max_tokens=self.max_tokens_to_generate,  
                    response_model=schema,  # Важно: передаем схему как response_model  
                    **api_extra_params  
                )  
                return response  # Instructor возвращает уже объект схемы  
            else:  
                # Обычный вызов для текстового ответа  
                response = client.chat.completions.create(  
                    model=self.model_name_original,  
                    messages=messages,  
                    max_tokens=self.max_tokens_to_generate,  
                    **api_extra_params  
                )  
                raw_response_content = response.choices[0].message.content  
                cleaned_response = self._clean_response(raw_response_content)  
                return cleaned_response  
        except Exception as e:  
            print(f"Error during synchronous generation for model '{self.model_name_original}', prompt '{prompt[:50]}...': {e}")  
            if schema is not None:  
                # Создаем заглушку с нулевыми значениями  
                default_values = {}  
                for field_name, field in schema.__annotations__.items():  
                    if field_name in ['clarity', 'depth', 'structure', 'relevance']:  
                        default_values[field_name] = 0.0  
                    else:  
                        default_values[field_name] = ""  
                return schema(**default_values)  
            return ""

    # --- ИЗМЕНЕННЫЙ МЕТОД a_generate ---
    async def a_generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Union[str, BaseModel]:  
        client = self.load_async_model()  
        call_args = self._prepare_api_call_args(prompt)  
        messages = call_args["messages"]  
        api_extra_params = call_args["api_extra_params"]  
    
        try:  
            # Если передана схема, используем response_model  
            if schema is not None:  
                response = await client.chat.completions.create(  
                    model=self.model_name_original,  
                    messages=messages,  
                    max_tokens=self.max_tokens_to_generate,  
                    response_model=schema,  # Важно: передаем схему как response_model  
                    **api_extra_params  
                )  
                return response  # Instructor возвращает уже объект схемы  
            else:  
                # Обычный вызов для текстового ответа  
                response = await client.chat.completions.create(  
                    model=self.model_name_original,  
                    messages=messages,  
                    max_tokens=self.max_tokens_to_generate,  
                    **api_extra_params  
                )  
                raw_response_content = response.choices[0].message.content  
                cleaned_response = self._clean_response(raw_response_content)  
                return cleaned_response  
        except Exception as e:  
            print(f"Error during asynchronous generation for model '{self.model_name_original}', prompt '{prompt[:50]}...': {e}")  
            if schema is not None:  
                # Создаем заглушку с нулевыми значениями  
                default_values = {}  
                for field_name, field in schema.__annotations__.items():  
                    if field_name in ['clarity', 'depth', 'structure', 'relevance']:  
                        default_values[field_name] = 0.0  
                    else:  
                        default_values[field_name] = ""  
                return schema(**default_values)  
            return ""

    def get_model_name(self) -> str:
        return self.model_name_original

In [ ]:
import re  
import os  
import json  
from typing import Optional, Union, Tuple, Dict  
from deepeval.models import DeepEvalBaseLLM  
from openai import OpenAI, AsyncOpenAI  
from pydantic import BaseModel  
from lmformatenforcer import JsonSchemaParser  
  
class SGlangModel(DeepEvalBaseLLM):  
    def __init__(self,   
                 model_name: str,   
                 base_url: str,   
                 api_key: Optional[str] = "NET",  
                 enable_thinking: bool = False):  
        """  
        Инициализирует модель SGlang.  
  
        Args:  
            model_name (str): Имя модели.  
            base_url (str): Базовый URL для API.  
            api_key (Optional[str]): API ключ. По умолчанию "NET".  
            enable_thinking (bool): Флаг для управления поведением моделей Qwen3.  
                                     Если True, к промпту для Qwen3 будет добавлен "/think".  
                                     Если False, к промпту для Qwen3 будет добавлен "/no_think".  
                                     По умолчанию False.  
        """  
        self.model_name = model_name  
        self.base_url = base_url  
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY")  
        self.enable_thinking = enable_thinking   
        self._sync_client: Optional[OpenAI] = None  
        self._async_client: Optional[AsyncOpenAI] = None  
  
    def load_model(self) -> OpenAI:  
        if self._sync_client is None:  
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)  
        return self._sync_client  
  
    def load_async_model(self) -> AsyncOpenAI:  
        if self._async_client is None:  
            self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)  
        return self._async_client  
  
    def _clean_qwen3_output(self, text_response: str) -> str:  
        """  
        Удаляет начальный блок <think>...</think> из ответов модели Qwen3.  
        """  
        pattern = r'^\s*<think>.*?</think>\s*'  
        cleaned_response = re.sub(pattern, '', text_response, count=1, flags=re.DOTALL)  
        return cleaned_response  
  
    def _trim_and_load_json(self, input_string: str) -> Dict:  
        """  
        Обрабатывает строку JSON, удаляя лишние символы и преобразуя в словарь.  
        """  
        start = input_string.find("{")  
        end = input_string.rfind("}") + 1  
        if end == 0 and start != -1:  
            input_string = input_string + "}"  
            end = len(input_string)  
        jsonStr = input_string[start:end] if start != -1 and end != 0 else ""  
        jsonStr = re.sub(r",\s*([\]}])", r"\1", jsonStr)  
        try:  
            return json.loads(jsonStr)  
        except json.JSONDecodeError:  
            error_str = "Модель вывела некорректный JSON. Пожалуйста, используйте более надежную модель."  
            raise ValueError(error_str)  
        except Exception as e:  
            raise Exception(f"Произошла непредвиденная ошибка: {str(e)}")  
  
    def generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Tuple[Union[str, BaseModel], float]:  
        """  
        Генерирует ответ от модели.   
        Для модели 'Qwen3' (без схемы):  
        - К промпту добавляется "/think" или "/no_think" в зависимости от self.enable_thinking.  
        - Блок <think> всегда удаляется из финального ответа.  
          
        Returns:  
            Tuple[Union[str, BaseModel], float]: Кортеж (результат, стоимость)  
        """  
        client = self.load_model()  
          
        processed_prompt = prompt  
        # Проверка на Qwen3 (нечувствительная к регистру) и отсутствие схемы  
        is_qwen3_text_mode = "qwen3" in self.model_name.lower() and schema is None  
  
        if is_qwen3_text_mode:  
            # Удаляем существующие теги /think или /no_think из конца промпта  
            processed_prompt = re.sub(r'\s*/think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()  
            processed_prompt = re.sub(r'\s*/no_think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()  
              
            if self.enable_thinking:  
                processed_prompt += " /think" # Инструктируем Qwen3 выполнить процесс размышления  
            else:  
                processed_prompt += " /no_think" # Инструктируем Qwen3 пропустить/минимизировать размышления  
          
        try:  
            if schema is None:  
                response = client.chat.completions.create(  
                    model=self.model_name,  
                    messages=[{"role": "user", "content": processed_prompt}],   
                )  
                raw_content = response.choices[0].message.content  
                  
                # Рассчитываем стоимость, если доступно  
                cost = 0  
                if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens'):  
                    cost = 0  # Замените на реальный расчет стоимости  
                  
                if is_qwen3_text_mode:  
                    # Для Qwen3 в текстовом режиме всегда очищаем вывод от блока <think>  
                    return self._clean_qwen3_output(raw_content), cost  
                else:  
                    # Для других моделей возвращаем "сырой" контент  
                    return raw_content, cost  
            else:  
                # Используем lm-format-enforcer через vLLM API  
                if "vllm" in self.base_url.lower():  
                    # Если используется vLLM, используем встроенную поддержку lm-format-enforcer  
                    response = client.chat.completions.create(  
                        model=self.model_name,  
                        messages=[{"role": "user", "content": processed_prompt}],  
                        extra_body={  
                            "guided_json": schema.model_json_schema(),  
                            "guided_decoding_backend": "lm-format-enforcer"  
                        }  
                    )  
                    raw_content = response.choices[0].message.content  
                      
                    # Рассчитываем стоимость, если доступно  
                    cost = 0  
                    if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens'):  
                        cost = 0  # Замените на реальный расчет стоимости  
                      
                    # Преобразуем JSON-строку в объект схемы  
                    json_data = self._trim_and_load_json(raw_content)  
                    return schema.model_validate(json_data), cost  
                else:  
                    # Для других API используем обычный запрос с последующей валидацией  
                    # Добавляем инструкцию для генерации JSON в соответствии со схемой  
                    schema_json = json.dumps(schema.model_json_schema(), indent=2)  
                    json_prompt = f"{processed_prompt}\n\nОтвет должен быть в формате JSON, соответствующем следующей схеме:\n{schema_json}\n\nВажно: Ответ должен содержать только валидный JSON без дополнительного текста."  
                      
                    response = client.chat.completions.create(  
                        model=self.model_name,  
                        messages=[{"role": "user", "content": json_prompt}],  
                    )  
                    raw_content = response.choices[0].message.content  
                      
                    # Рассчитываем стоимость, если доступно  
                    cost = 0  
                    if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens'):  
                        cost = 0  # Замените на реальный расчет стоимости  
                      
                    # Обрабатываем и валидируем JSON  
                    json_data = self._trim_and_load_json(raw_content)  
                    return schema.model_validate(json_data), cost  
        except Exception as e:  
            print(f"Ошибка при синхронной генерации для промпта '{prompt[:50]}...': {e}")  
            raise e  
  
    async def a_generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Tuple[Union[str, BaseModel], float]:  
        """  
        Асинхронно генерирует ответ от модели.  
        Для модели 'Qwen3' (без схемы):  
        - К промпту добавляется "/think" или "/no_think" в зависимости от self.enable_thinking.  
        - Блок <think> всегда удаляется из финального ответа.  
          
        Returns:  
            Tuple[Union[str, BaseModel], float]: Кортеж (результат, стоимость)  
        """  
        client = self.load_async_model()  
  
        processed_prompt = prompt  
        # Проверка на Qwen3 (нечувствительная к регистру) и отсутствие схемы  
        is_qwen3_text_mode = "qwen3" in self.model_name.lower() and schema is None  
  
        if is_qwen3_text_mode:  
            # Удаляем существующие теги /think или /no_think из конца промпта  
            processed_prompt = re.sub(r'\s*/think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()  
            processed_prompt = re.sub(r'\s*/no_think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()  
  
            if self.enable_thinking:  
                processed_prompt += " /think" # Инструктируем Qwen3 выполнить процесс размышления  
            else:  
                processed_prompt += " /no_think" # Инструктируем Qwen3 пропустить/минимизировать размышления  
  
        try:  
            if schema is None:  
                response = await client.chat.completions.create(  
                    model=self.model_name,  
                    messages=[{"role": "user", "content": processed_prompt}],   
                )  
                raw_content = response.choices[0].message.content  
                  
                # Рассчитываем стоимость, если доступно  
                cost = 0  
                if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens'):  
                    cost = 0  # Замените на реальный расчет стоимости  
  
                if is_qwen3_text_mode:  
                    # Для Qwen3 в текстовом режиме всегда очищаем вывод от блока <think>  
                    return self._clean_qwen3_output(raw_content), cost  
                else:  
                    # Для других моделей возвращаем "сырой" контент  
                    return raw_content, cost  
            else:  
                # Используем lm-format-enforcer через vLLM API  
                if "vllm" in self.base_url.lower():  
                    # Если используется vLLM, используем встроенную поддержку lm-format-enforcer  
                    response = await client.chat.completions.create(  
                        model=self.model_name,  
                        messages=[{"role": "user", "content": processed_prompt}],  
                        extra_body={  
                            "guided_json": schema.model_json_schema(),  
                            "guided_decoding_backend": "lm-format-enforcer"  
                        }  
                    )  
                    raw_content = response.choices[0].message.content  
                      
                    # Рассчитываем стоимость, если доступно  
                    cost = 0  
                    if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens'):  
                        cost = 0  # Замените на реальный расчет стоимости  
                      
                    # Преобразуем JSON-строку в объект схемы  
                    json_data = self._trim_and_load_json(raw_content)  
                    return schema.model_validate(json_data), cost  
                else:  
                    # Для других API используем обычный запрос с последующей валидацией  
                    # Добавляем инструкцию для генерации JSON в соответствии со схемой  
                    schema_json = json.dumps(schema.model_json_schema(), indent=2)  
                    json_prompt = f"{processed_prompt}\n\nОтвет должен быть в формате JSON, соответствующем следующей схеме:\n{schema_json}\n\nВажно: Ответ должен содержать только валидный JSON без дополнительного текста."  
                      
                    response = await client.chat.completions.create(  
                        model=self.model_name,  
                        messages=[{"role": "user", "content": json_prompt}],  
                    )  
                    raw_content = response.choices[0].message.content  
                      
                    # Рассчитываем стоимость, если доступно  
                    cost = 0  
                    if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens'):  
                        cost = 0  # Замените на реальный расчет стоимости  
                      
                    # Обрабатываем и валидируем JSON  
                    json_data = self._trim_and_load_json(raw_content)  
                    return schema.model_validate(json_data), cost  
        except Exception as e:  
            print(f"Ошибка при асинхронной генерации для промпта '{prompt[:50]}...': {e}")  
            raise e  
  
    def get_model_name(self) -> str:  
        return self.model_name

In [4]:
import os
import re
from typing import Optional, List, Dict, Any
from deepeval.models import DeepEvalBaseLLM
from openai import OpenAI, AsyncOpenAI

class SGlangModel(DeepEvalBaseLLM):
    """
    Generic DeepEval LLM wrapper for models served via an
    OpenAI-compatible API (e.g., SGLang, vLLM).
    Can attempt to enable model-specific thinking/reasoning modes
    based on model_name and removes <think>...</think> tags from responses if they appear.
    """
    # Специфическая инструкция для Cogito
    COGITO_THINKING_INSTRUCTION = "Enable deep thinking subroutine."

    def __init__(self,
                 model_name: str, # Имя модели, используется для логики
                 base_url: str,   # URL эндпоинта OpenAI-совместимого API
                 api_key: Optional[str] = None, # API ключ (часто "EMPTY" или не нужен для локальных)
                 attempt_thinking_mode: bool = False, # Пытаться ли включить режим рассуждений?
                 cleaning_method: str = "rsplit", # Метод очистки тегов: 'rsplit' или 'regex'
                 max_tokens: int = 8192 # Макс. токенов для генерации
                ):
        """
        Initializes the SGlangModel wrapper.

        Args:
            model_name: Name of the model being served (e.g., "Qwen/Qwen3-30B-A3B", "deepcogito/cogito-v1...").
            base_url: The base URL of the OpenAI-compatible API endpoint.
            api_key: Optional API key for the endpoint. Defaults to env variable or "EMPTY".
            attempt_thinking_mode: If True, tries to enable thinking mode based on model_name. Defaults to False.
            cleaning_method: Method to use for removing <think> tags ('rsplit' or 'regex'). Defaults to 'rsplit'.
            max_tokens: Maximum number of new tokens to generate. Defaults to 8192.
        """
        self.model_name_original = model_name # Сохраняем оригинальное имя
        self.model_name_lower = model_name.lower() # Сохраняем в нижнем регистре для сравнения
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY", "EMPTY")
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None
        self.attempt_thinking_mode = attempt_thinking_mode
        self.cleaning_method = cleaning_method
        self.max_tokens_to_generate = max_tokens

    def load_model(self) -> OpenAI:
        """Loads or returns the synchronous OpenAI client."""
        if self._sync_client is None:
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._sync_client

    def load_async_model(self) -> AsyncOpenAI:
        """Loads or returns the asynchronous OpenAI client."""
        if self._async_client is None:
            self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._async_client

    def _clean_response(self, raw_response: str) -> str:
        """
        Helper function to remove <think>...</think> blocks from the raw response string.
        Uses the method specified in self.cleaning_method.
        """
        if not raw_response:
            return ""

        if self.cleaning_method == "rsplit":
            closing_tag = '</think>'
            if closing_tag in raw_response:
                # Разделяем по последнему тегу и берем правую часть
                main_answer = raw_response.rsplit(closing_tag, 1)[-1].strip()
            else:
                # Если тега нет, просто убираем пробелы по краям
                main_answer = raw_response.strip()
            return main_answer

        elif self.cleaning_method == "regex":
            # Шаблон для удаления <think>...</think> и пробелов после
            pattern = r'<think>.*?</think>\s*'
            # Заменяем найденное на пустую строку, DOTALL для переносов строк
            main_answer = re.sub(pattern, '', raw_response, flags=re.DOTALL).strip()
            return main_answer
        else:
            # Если метод не 'rsplit' и не 'regex', возвращаем как есть, убрав пробелы
            print(f"Warning: Unknown cleaning_method '{self.cleaning_method}'. Returning raw response.")
            return raw_response.strip()

    def _prepare_api_call_args(self, prompt: str) -> Dict[str, Any]:
        """
        Prepares the messages list and a dictionary of extra API parameters
        based on the model name and the attempt_thinking_mode flag.
        """
        # Базовый список сообщений - только промпт пользователя
        messages: List[Dict[str, str]] = [{"role": "user", "content": prompt}]
        # Словарь для дополнительных параметров API (например, enable_thinking)
        api_extra_params: Dict[str, Any] = {}

        if self.attempt_thinking_mode:
            # --- Логика для конкретных моделей ---
            if "cogito" in self.model_name_lower:
                # Добавляем системный промпт для Cogito В НАЧАЛО списка
                messages.insert(0, {"role": "system", "content": self.COGITO_THINKING_INSTRUCTION})
                print(f"Info: Enabling Cogito thinking mode via system prompt for model '{self.model_name_original}'.")

            elif "qwen3" in self.model_name_lower:
                # Добавляем параметр API для Qwen3
                # ВАЖНО: Имя параметра 'enable_thinking' является ПРЕДПОЛОЖЕНИЕМ.
                # Проверьте документацию вашего сервера SGLang/vLLM!
                # Если возникнет ошибка, возможно, потребуется использовать 'extra_body'.
                api_extra_params["enable_thinking"] = True
                print(f"Info: Attempting to enable Qwen3 thinking mode via API parameter for model '{self.model_name_original}'.")

            else:
                # Модель не распознана для специальной обработки режима рассуждений
                print(f"Warning: 'attempt_thinking_mode' is True, but no specific handling defined for model '{self.model_name_original}'. Making standard call.")
            # --- Конец логики для моделей ---

        # Возвращаем словарь с подготовленными сообщениями и доп. параметрами
        return {"messages": messages, "api_extra_params": api_extra_params}

    def generate(self, prompt: str) -> str:
        """
        Generates a response synchronously.
        If attempt_thinking_mode is True, it modifies the request based on the model.
        It always cleans the response to remove <think> tags.
        """
        client = self.load_model()
        # Подготавливаем аргументы для вызова API
        call_args = self._prepare_api_call_args(prompt)
        messages = call_args["messages"]
        api_extra_params = call_args["api_extra_params"]

        try:
            # Выполняем вызов API, передавая основные и дополнительные параметры
            response = client.chat.completions.create(
                model=self.model_name_original, # Используем оригинальное имя модели
                messages=messages,
                max_tokens=self.max_tokens_to_generate,
                **api_extra_params # Распаковываем доп. параметры (может быть пустым)
                # Примечание: Если 'enable_thinking' не работает как прямой параметр,
                # попробуйте передать его так (закомментировав строку выше с **api_extra_params):
                # extra_body=api_extra_params if api_extra_params else None
            )
            # Получаем сырой текстовый ответ
            raw_response_content = response.choices[0].message.content
            # Всегда очищаем ответ от тегов <think>
            cleaned_response = self._clean_response(raw_response_content)
            return cleaned_response
        except Exception as e:
            print(f"Error during synchronous generation for model '{self.model_name_original}', prompt '{prompt[:50]}...': {e}")
            # Добавляем подсказки, если ошибка связана с параметром 'enable_thinking'
            if api_extra_params.get("enable_thinking") and "unexpected keyword argument 'enable_thinking'" in str(e).lower():
                 print("Hint: The API might not support 'enable_thinking' as a direct parameter. Try modifying the wrapper to use 'extra_body'.")
            elif api_extra_params.get("enable_thinking") and "extra_body" in str(e).lower():
                 print("Hint: Check the exact API documentation for SGLang/vLLM OpenAI-compatible endpoint for enabling Qwen3 thinking mode.")
            return "" # Возвращаем пустую строку в случае ошибки

    async def a_generate(self, prompt: str) -> str:
        """
        Generates a response asynchronously.
        If attempt_thinking_mode is True, it modifies the request based on the model.
        It always cleans the response to remove <think> tags.
        """
        client = self.load_async_model()
        # Подготавливаем аргументы для вызова API
        call_args = self._prepare_api_call_args(prompt)
        messages = call_args["messages"]
        api_extra_params = call_args["api_extra_params"]

        try:
            # Выполняем асинхронный вызов API
            response = await client.chat.completions.create(
                model=self.model_name_original, # Используем оригинальное имя модели
                messages=messages,
                max_tokens=self.max_tokens_to_generate,
                **api_extra_params # Распаковываем доп. параметры
                 # Примечание: Если 'enable_thinking' не работает как прямой параметр,
                 # попробуйте передать его так (закомментировав строку выше с **api_extra_params):
                 # extra_body=api_extra_params if api_extra_params else None
            )
            # Получаем сырой текстовый ответ
            raw_response_content = response.choices[0].message.content
            # Всегда очищаем ответ от тегов <think>
            cleaned_response = self._clean_response(raw_response_content)
            return cleaned_response
        except Exception as e:
            print(f"Error during asynchronous generation for model '{self.model_name_original}', prompt '{prompt[:50]}...': {e}")
            # Добавляем аналогичные подсказки про enable_thinking / extra_body
            if api_extra_params.get("enable_thinking") and "unexpected keyword argument 'enable_thinking'" in str(e).lower():
                 print("Hint: The API might not support 'enable_thinking' as a direct parameter. Try modifying the wrapper to use 'extra_body'.")
            elif api_extra_params.get("enable_thinking") and "extra_body" in str(e).lower():
                 print("Hint: Check the exact API documentation for SGLang/vLLM OpenAI-compatible endpoint for enabling Qwen3 thinking mode.")
            return "" # Возвращаем пустую строку в случае ошибки

    def get_model_name(self) -> str:
        """
        Returns the original name of the model used for initialization.
        """
        # Возвращаем оригинальное имя, сохраненное при инициализации
        return self.model_name_original


In [5]:
CogitoLLM= SGlangModel(model_name="deepcogito/cogito-v1-preview-llama-8B", base_url="http://85.143.167.11:30000/v1")

# Qwen3_30_Reasoning = LLMModel(model_name="qwen3-30-lmstudio", base_url="http://85.143.167.11:30000/v1", attempt_thinking_mode=True, cleaning_method="rsplit")
# Qwen3_30 = LLMModel(model_name="qwen3-30-lmstudio", base_url="http://85.143.167.11:30000/v1", attempt_thinking_mode=False)

# Qwen3_32_Reasoning = LLMModel(model_name="Qwen/Qwen3-32B-AWQ", base_url="http://85.143.167.11:30000/v1", attempt_thinking_mode=True, cleaning_method="rsplit")
# Qwen3_32 = LLMModel(model_name="Qwen/Qwen3-32B-AWQ", base_url="http://85.143.167.11:30000/v1", attempt_thinking_mode=False)

# Qwen3_8_Reasoning = LLMModel(model_name="Qwen/Qwen3-8B-FP8", base_url="http://85.143.167.11:30000/v1", attempt_thinking_mode=True, cleaning_method="rsplit")
# Qwen3_8 = LLMModel(model_name="Qwen/Qwen3-8B-FP8", base_url="http://85.143.167.11:30000/v1", attempt_thinking_mode=False)



In [6]:
BertaEmbeddings = InfinityEmbeddingModel(model_name="sergeyzh/BERTA", base_url="http://127.0.0.1:7997")
USER2Embeddings = InfinityEmbeddingModel(model_name="deepvk/USER2-base", base_url="http://127.0.0.1:7997")
RuEnROBERTAEmbeddings = InfinityEmbeddingModel(model_name="ai-forever/ru-en-RoSBERTa", base_url="http://127.0.0.1:7997")

In [7]:
from deepeval.synthesizer import Synthesizer, Evolution  
from deepeval.synthesizer.config import StylingConfig, EvolutionConfig, ContextConstructionConfig, FiltrationConfig  
  
context_construction_config = ContextConstructionConfig(  
    chunk_size=2048,                 
    chunk_overlap=0,            
    max_contexts_per_document=3,
    context_quality_threshold=0.4,
    max_retries=2,
    critic_model = CogitoLLM,
    embedder = BertaEmbeddings)  
  
styling_config = StylingConfig(  
    input_format="""Сложные академические вопросы на русском языке, основанные на содержании медицинских учебников.   
    Вопросы должны быть сформулированы с использованием точной медицинской терминологии,   
    требовать глубокого понимания материала и проверять способность применять теоретические знания   
    к клиническим ситуациям.""",  
      
    expected_output_format="""Структурированные, научно обоснованные ответы на русском языке,   
    включающие ключевые концепции из учебника, точные определения, классификации и механизмы.   
    Ответы должны быть организованы в логической последовательности с использованием подзаголовков, где это уместно.""",  
      
    task="""Создание высококачественных экзаменационных вопросов на русском языке,   
    которые точно отражают содержание учебника и проверяют глубину понимания материала.""",  
      
    scenario="""Преподаватель медицинского вуза создает банк вопросов для экзаменов,   
    тестирования и самопроверки студентов. Вопросы должны точно соответствовать   
    содержанию учебников и быть пригодными для оценки компетенций студентов   
    разных курсов медицинского образования.""",  
)  
  
evolution_config = EvolutionConfig(  
    num_evolutions=2, 
    evolutions={  
        Evolution.CONCRETIZING: 0.25,     
        Evolution.MULTICONTEXT: 0.25,   
        Evolution.COMPARATIVE: 0.25,   
        Evolution.CONSTRAINED: 0.25,
    }  
) 

filtration_config = FiltrationConfig(  
    synthetic_input_quality_threshold=0.5,
    critic_model= CogitoLLM,
    max_quality_retries=2                
)

synthesizer = Synthesizer(
    model=CogitoLLM,  
    styling_config=styling_config,  
    # evolution_config=evolution_config,
    # filtration_config=filtration_config,  
    async_mode=True,  
    max_concurrent=8
)  
  


In [8]:
goldens = await synthesizer.a_generate_goldens_from_docs(  
    document_paths=['/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Anatomia_cheloveka_1_tom_2-52-57.pdf',
                    '/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Kapandzhi_-_Pozvonochnik-276-284.pdf'],  
    include_expected_output=True,  
    max_goldens_per_context=3,  
    context_construction_config=context_construction_config  
)  
  

✨ 🧩 ✨ Generating Contexts:   0%|          | 0/12 [00:00<?, ?it/s]


ValueError: Evaluation LLM outputted an invalid JSON. Please use a better evaluation model.